In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

# Chapter 11. Training Deep Neural Networks

# Exercise 8

In [2]:
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [3]:
# TODO this?
#x_train_full = x_train_full / 255.0
#x_test = x_test / 255.0

In [4]:
x_val = x_train_full[:5000]
y_val = y_train_full[:5000]
x_train = x_train_full[5000:]
y_train = y_train_full[5000:]

### Part a and b

In [9]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(32, 32, 3)))
for _ in range(20):
    model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=3e-4)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)
run_index = 5 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_no_norm_{:03d}".format(run_index))
#tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

class LearningRateTensorBoard(keras.callbacks.TensorBoard):
    # add other arguments to __init__ if you need
    def __init__(self, log_dir, **kwargs):
        super().__init__(log_dir=log_dir, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs.update({'lr': keras.backend.eval(self.model.optimizer.lr)})
        super().on_epoch_end(epoch, logs)
tensorboard_cb = LearningRateTensorBoard(run_logdir)

callbacks = [
    #early_stopping_cb,
    model_checkpoint_cb,
    tensorboard_cb]

model.fit(x_train, y_train, epochs=20,
          validation_data=(x_val, y_val),
          callbacks=callbacks)

Epoch 1/20
1407/1407 [==============================] - 11s 6ms/step - loss: 2.8284 - accuracy: 0.2020 - val_loss: 2.0970 - val_accuracy: 0.2400 - lr: 3.0000e-04
Epoch 2/20
1407/1407 [==============================] - 8s 6ms/step - loss: 1.9397 - accuracy: 0.2820 - val_loss: 2.4095 - val_accuracy: 0.1984 - lr: 3.0000e-04
Epoch 3/20
1407/1407 [==============================] - 8s 6ms/step - loss: 1.8631 - accuracy: 0.3174 - val_loss: 1.9118 - val_accuracy: 0.2940 - lr: 3.0000e-04
Epoch 4/20
1407/1407 [==============================] - 8s 6ms/step - loss: 1.8091 - accuracy: 0.3408 - val_loss: 2.3432 - val_accuracy: 0.2238 - lr: 3.0000e-04
Epoch 5/20
1407/1407 [==============================] - 8s 6ms/step - loss: 1.7660 - accuracy: 0.3582 - val_loss: 1.7345 - val_accuracy: 0.3714 - lr: 3.0000e-04
Epoch 6/20
1407/1407 [==============================] - 8s 6ms/step - loss: 1.7167 - accuracy: 0.3804 - val_loss: 1.7449 - val_accuracy: 0.3778 - lr: 3.0000e-04
Epoch 7/20
1407/1407 [===========

### Part c

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_bn_model.h5")
model.evaluate(X_valid, y_valid)